In [63]:
!mkdir data
!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
!mkdir write


In [64]:
import numpy as np
import pandas as pd
import scanpy as sc
import leidenalg

In [65]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')


In [66]:
results_file = 'write/pbmc3k.h5ad'  # the file that will store the analysis results


In [67]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading


In [68]:
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`


In [69]:
adata

In [70]:
sc.pl.highest_expr_genes(adata, n_top=20, )


In [71]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)


In [72]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


In [73]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)


In [74]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')


In [75]:
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]


In [76]:
sc.pp.normalize_total(adata, target_sum=1e4)


In [77]:
sc.pp.log1p(adata)


In [78]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)


In [79]:
sc.pl.highly_variable_genes(adata)


In [80]:
adata.raw = adata


In [81]:
adata = adata[:, adata.var.highly_variable]


In [82]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])


In [83]:
sc.pp.scale(adata, max_value=10)


In [84]:
sc.tl.pca(adata, svd_solver='arpack')


In [85]:
sc.pl.pca(adata, color='CST3')


In [86]:
sc.pl.pca_variance_ratio(adata, log=True)


In [87]:
adata.write(results_file)


In [88]:
adata


In [89]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)


In [90]:
sc.tl.leiden(adata)
sc.tl.paga(adata)
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')


In [91]:
sc.tl.umap(adata)


In [92]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])


In [93]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'], use_raw=False)


In [94]:
sc.tl.leiden(adata)


In [95]:
sc.pl.umap(adata, color=['leiden', 'CST3', 'NKG7'])


In [96]:
adata.write(results_file)


In [97]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)


In [98]:
sc.settings.verbosity = 2  # reduce the verbosity


In [99]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)


In [100]:
adata.write(results_file)


In [101]:
sc.tl.rank_genes_groups(adata, 'leiden', method='logreg')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)


In [102]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']


In [103]:
adata = sc.read(results_file)


In [104]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)


In [105]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)


In [106]:
sc.tl.rank_genes_groups(adata, 'leiden', groups=['0'], reference='1', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groups=['0'], n_genes=20)


In [107]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)


In [108]:
adata = sc.read(results_file)


In [109]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)


In [110]:
sc.pl.violin(adata, ['CST3', 'NKG7', 'PPBP'], groupby='leiden')


In [111]:
new_cluster_names = [
    'CD4 T', 'CD14 Monocytes',
    'B', 'CD8 T',
    'NK', 'FCGR3A Monocytes',
    'Dendritic', 'Megakaryocytes']
adata.rename_categories('leiden', new_cluster_names)


In [112]:
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='', frameon=False, save='.pdf')


In [113]:
sc.pl.dotplot(adata, marker_genes, groupby='leiden');


In [114]:
sc.pl.stacked_violin(adata, marker_genes, groupby='leiden', rotation=90);


In [115]:
adata


In [116]:
#set biosample_id value
#biosample = np.random.choice(["mm1_lymph", "hu1_blood", "mm1_blood", "mm2_blood"], size=(adata.n_obs,))
adata.obs["biosample_id"] = "hu1_blood"

#set donor_id value
adata.obs["donor_id"] = "human1"

#set cell_type and cell_type__ontology_label
leiden_list = []
for i in adata.obs["leiden"]:
    leiden_list.append(i)

#leiden categories
leiden_cell = ['CD4 T', 'CD14 Monocytes', 'B', 'CD8 T', 'NK', 'FCGR3A Monocytes', 'Dendritic', 'Megakaryocytes']

#cell_type ontology labels corresponding with above
cell_type_ot = ["CL_0000492", "CL_0001054", "CL_0000236", "CL_0000625", "CL_0000814", "CL_0000860", "CL_0000451", "CL_0000556"]

leiden_to_cell_type = dict(zip(leiden_cell, cell_type_ot))

cell_type_list = []
for i in leiden_list:
    cell_type_list.append(leiden_to_cell_type[i])

adata.obs["cell_type"] = cell_type_list


#cell_type__ontology_label corresponding with above
cell_type_ot_label = ["CD4-positive helper T cell", "CD14-positive monocyte", "B cell", "CD8-positive, alpha-beta T cell", "mature NK T cell", "classical monocyte", "dendritic cell", "megakaryocyte"]

leiden_to_cell_ot = dict(zip(leiden_cell, cell_type_ot_label))

cell_type_ot_list = []
for i in leiden_list:
    cell_type_ot_list.append(leiden_to_cell_ot[i])

adata.obs["cell_type__ontology_label"] = cell_type_ot_list

#set sex value
adata.obs["sex"] = "female"

#set average_intensity value
size_intensity = adata.n_obs
random_intensities = np.random.uniform(-20, 20, size_intensity)
rounded_list = []
for i in random_intensities:
    rounded_list.append(round(i, 3))
    
adata.obs["average_intensity"] = rounded_list

#set species
adata.obs["species"] = "NCBITaxon_9606"

#set species__ontology
adata.obs["species__ontology_label"] = "Homo sapiens"

#set disease label
adata.obs["disease"] = "PATO_0000461"

#set disease__ontology_label
adata.obs["disease__ontology_label"] = "normal"

#set organ label
adata.obs["organ"] = "UBERON_0000178"


#set organ ontology label
adata.obs["organ__ontology_label"] = "blood"

#set library prepartion protocol
adata.obs["library_preparation_protocol"] = "EFO_0009899"

#set library preparation ontology
adata.obs["library_preparation_protocol__ontology_label"] = "10X 3' v2"


In [117]:
df = sc.get.obs_df(adata, keys = ['biosample_id', 'donor_id', 'cell_type', 'cell_type__ontology_label', 'sex', 'average_intensity', 'species', 'species__ontology_label', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label'])
df

In [118]:
adata.write_h5ad("test_hdf5_file")